In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('testApp').getOrCreate()

21/12/28 07:12:31 WARN Utils: Your hostname, Home-Laptop-T430 resolves to a loopback address: 127.0.1.1; using 192.168.0.23 instead (on interface wlp3s0)
21/12/28 07:12:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/28 07:12:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [21]:
df_pyspark = spark.read.options(header=True, inferSchema = True)\
.csv('/media/kalyan/D/workspace/Datasets/tips.csv')

In [26]:
df_pyspark.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [27]:
print((df_pyspark.count(), len(df_pyspark.columns)))

(244, 7)


In [28]:
from pyspark.sql.functions import isnan, when, count, col

df_pyspark.select([count(when(isnan(c), c)).alias(c) for c in df_pyspark.columns]).show()

+----------+---+---+------+---+----+----+
|total_bill|tip|sex|smoker|day|time|size|
+----------+---+---+------+---+----+----+
|         0|  0|  0|     0|  0|   0|   0|
+----------+---+---+------+---+----+----+



In [31]:
from pyspark.ml.feature import StringIndexer

In [32]:
indexer = StringIndexer(inputCol='sex', outputCol='sex_indexed')
df_pyspark_new = indexer.fit(df_pyspark).transform(df_pyspark)
df_pyspark_new.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [33]:
indexer = StringIndexer(inputCols=['smoker','day','time'], outputCols=['smoker_indexed','day_indexed','time_indexed'])
df_pyspark_final = indexer.fit(df_pyspark_new).transform(df_pyspark_new)
df_pyspark_final.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [35]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols=['total_bill', 'sex_indexed', 'smoker_indexed','day_indexed','time_indexed','size'],outputCol='Input Features')

In [37]:
output = featureAssembler.transform(df_pyspark_final)

In [38]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|      Input Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[16.99,1.0,0.0,1....|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[10.34,0.0,0.0,1....|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[21.01,0.0,0.0,1....|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[23.68,0.0,0.0,1....|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[24.59,1.0,0.0,1....|
|     25.29|4.71|  Male|    No|S

In [39]:
dataset = output.select(['Input Features', 'tip'])

In [40]:
dataset.show()

+--------------------+----+
|      Input Features| tip|
+--------------------+----+
|[16.99,1.0,0.0,1....|1.01|
|[10.34,0.0,0.0,1....|1.66|
|[21.01,0.0,0.0,1....| 3.5|
|[23.68,0.0,0.0,1....|3.31|
|[24.59,1.0,0.0,1....|3.61|
|[25.29,0.0,0.0,1....|4.71|
|[8.77,0.0,0.0,1.0...| 2.0|
|[26.88,0.0,0.0,1....|3.12|
|[15.04,0.0,0.0,1....|1.96|
|[14.78,0.0,0.0,1....|3.23|
|[10.27,0.0,0.0,1....|1.71|
|[35.26,1.0,0.0,1....| 5.0|
|[15.42,0.0,0.0,1....|1.57|
|[18.43,0.0,0.0,1....| 3.0|
|[14.83,1.0,0.0,1....|3.02|
|[21.58,0.0,0.0,1....|3.92|
|[10.33,1.0,0.0,1....|1.67|
|[16.29,0.0,0.0,1....|3.71|
|[16.97,1.0,0.0,1....| 3.5|
|(6,[0,5],[20.65,3...|3.35|
+--------------------+----+
only showing top 20 rows



In [41]:
from pyspark.ml.regression import LinearRegression